In [1]:
%run init.ipynb

matchzoo version 1.1.1
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
/Users/yyang/.matchzoo/datasets/wiki_qa/WikiQACorpus
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.BertKNRM.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing length_right with len: 100%|██████████| 2300/2300 [00:00<00:00, 728782.90it/s]


In [4]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
    #resample=True,
    #sort=False,
    #batch_size=20,
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
    #batch_size=20,
)

In [5]:
padding_callback = mz.models.BertKNRM.get_default_padding_callback()
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    stage='train',
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    stage='dev',
    callback=padding_callback
)

In [6]:
model = mz.models.BertKNRM()

model.params['task'] = ranking_task
model.params['mode'] = 'bert-base-uncased'
model.params['dropout_rate'] = 0.2
model.params['kernel_num'] = 21

model.build()
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

Trainable params:  218964502


In [7]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 5e-5},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

from pytorch_transformers import AdamW, WarmupLinearSchedule

optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, betas=(0.9, 0.98), eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=6, t_total=-1)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=5
)

In [9]:
trainer.run()

[Iter-64 Loss-0.980]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.3312 - normalized_discounted_cumulative_gain@5(0.0): 0.4207 - mean_average_precision(0.0): 0.3749



[Iter-128 Loss-0.604]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5983 - normalized_discounted_cumulative_gain@5(0.0): 0.6389 - mean_average_precision(0.0): 0.6062



[Iter-192 Loss-0.336]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6525 - normalized_discounted_cumulative_gain@5(0.0): 0.7144 - mean_average_precision(0.0): 0.6626



[Iter-256 Loss-0.132]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7081 - normalized_discounted_cumulative_gain@5(0.0): 0.7571 - mean_average_precision(0.0): 0.6937



[Iter-320 Loss-0.055]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.6776 - normalized_discounted_cumulative_gain@5(0.0): 0.7278 - mean_average_precision(0.0): 0.6771

Cost time: 13540.879316329956s


In [12]:
torch.tensor([[]]).shape

torch.Size([1, 0])

In [11]:
import torch.nn as nn
m = nn.Linear(21, 1)
input = torch.randn(100, 4, 21)
output = m(input)
print(output.size())

torch.Size([100, 4, 1])


In [5]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_input = tokenizer(query, return_tensors='pt')
output = model(**encoded_input)

TypeError: forward() got an unexpected keyword argument 'input_ids'

In [11]:
encoded_input

{'input_ids': tensor([[  101,  3919, 21003,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [16]:
model.bert_q(torch.tensor([[  101,  3919, 21003,   102], [1,2,3,4]]).long(), torch.tensor([[]]).long())[0].shape

RuntimeError: Sizes of tensors must match except in dimension 1. Got 2 and 1 in dimension 0 (The offending index is 1)

In [15]:
model.bert_q(encoded_input['input_ids'].long(), torch.tensor([[]]).long())[1].shape

torch.Size([1, 768])

In [9]:
torch.__version__

'1.6.0'